# Análisis de satisfacción para aerolinea

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn import ensemble
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import warnings
from  pandasgui import show

In [ ]:
df = pd.read_csv('../data/airline_passenger_satisfaction.csv')

pd.set_option("display.max_columns", None)
warnings.filterwarnings("ignore")
df.head(10)

In [ ]:
df.info()

## Interpretación de datos

- **Gender**: Femenino o masculino  
- **Customer type**: Cliente de aerolinea (regular o no regular)   
- **Age**: Edad del pasajero  
- **Type of travel**: El propósito del vuelo del pasajero (viaje personal o de negocios)  
- **Class**: business, economy, economy plus  
- **Flight distance**: Distancia de vuelo  
- **Inflight wifi service**: Nivel de satisfacción con el servicio wifi a bordo (0: no puntuado; 1-5)  
- **Departure/Arrival time convenient**: Nivel de satisfacción de la hora de salida/llegada (0: no calificado; 1-5) Facilidad de reserva  
- **Ease of Online booking**: Tasa de satisfacción de reservas online (0: no puntuado; 1-5)  
- **Gate location**: Nivel de satisfacción con la ubicación de la puerta (0: no calificado; 1-5)  
- **Food and drink**: Nivel de satisfacción con alimentos y bebidas (0: no calificado; 1-5)  
- **Online boarding**: Nivel de satisfacción con el embarque online (0: no puntuado; 1-5)  
- **Seat comfort**: Nivel de satisfacción del asiento (0: no clasificado; 1-5)  
- **Inflight entertainment**: Satisfacción con el entretenimiento durante el vuelo (0: no clasificado; 1-5)  
- **On-board service**: Nivel de satisfacción con el servicio a bordo (0: no calificado; 1-5)  
- **Leg room service**: Nivel de satisfacción con el servicio de sala de piernas (0: no calificado; 1-5)  
- **Baggage handling**: Nivel de satisfacción con el manejo de equipaje (0: no calificado; 1-5)  
- **Checkin service**: Nivel de satisfacción con el servicio de check-in (0: no calificado; 1-5)  
- **Inflight service**: Nivel de satisfacción con el servicio durante el vuelo (0: no calificado; 1-5)  
- **Cleanliness**: Nivel de satisfacción con la limpieza (0: no puntuado; 1-5)

In [ ]:
df.isnull().sum()

In [ ]:
# function delete_columns
df = df.drop("Unnamed: 0",axis=1)
df = df.drop("id",axis=1)

df

In [ ]:
# impute_null
df['Arrival Delay in Minutes'].fillna(df['Arrival Delay in Minutes'].median(axis = 0), inplace = True)

In [ ]:
categoricas = ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'satisfaction', 'Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location','Food and drink', 'Online boarding','Inflight entertainment', 'On-board service', 'Leg room service','Baggage handling', 'Checkin service', 'Inflight service','Cleanliness']
categoricas_one = ['Gender', 'Customer Type', 'Type of Travel', 'Class', 'satisfaction']

In [ ]:
# function convert_objetive_bool
df.loc[df['satisfaction'] == "satisfied", 'satisfaction'] = True
df.loc[df['satisfaction'] == "neutral or dissatisfied", 'satisfaction'] = False


In [ ]:
encoder = OneHotEncoder(sparse_output=False)
dfCodificador = encoder.fit_transform(df[categoricas_one])
columnasNuevas = encoder.get_feature_names_out(categoricas_one)
dfcodificado = pd.DataFrame(dfCodificador, columns= columnasNuevas)

df = pd.concat([df, dfcodificado], axis=1)
df.drop(categoricas_one, axis=1, inplace=True)

df

In [ ]:
df = df.drop(["satisfaction_False", "Type of Travel_Personal Travel", "Gender_Male", "Customer Type_disloyal Customer"], axis=1)

In [ ]:
df = df.rename(columns={"satisfaction_True" : "Satisfied", "Type of Travel_Business travel" : "Business travel", "Gender_Female" : "Gender", "Customer Type_Loyal Customer" : "Loyal Customer"})

#En la columna Gender, se divide en Famale y Male, para Female es 1 y Male es 0

In [ ]:
df

In [ ]:
#Escalado de columnas numericas

from sklearn.preprocessing import MinMaxScaler

columns_to_scale = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']

scaler = MinMaxScaler()
df[columns_to_scale] = scaler.fit_transform(df[columns_to_scale])

In [ ]:
#Modelo de Random Forest

#importamos librerias
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

from matplotlib import rcParams
import warnings

warnings.filterwarnings("ignore")

# tamaño de la figura en pulgadas
rcParams["figure.figsize"] = 10, 6
np.random.seed(42)

In [ ]:
#separamos nuestro variable X y nuestra variable Y que utilizara el modelo 
X = df.drop("Satisfied", axis=1)
y = df["Satisfied"]

In [ ]:
#Separacion de de datos de entrenamiento y de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.20, random_state=42)

In [ ]:
#Creacion y entrenamiento del Random Forest
classifier = RandomForestClassifier(n_estimators=3)
classifier.fit(X_train,y_train)

In [ ]:

#Prediccion de nuestros datos de prueba
y_pred = classifier.predict(X_test)

In [ ]:
#Accuracy de nuestro modelo
print("Precisión:", accuracy_score(y_test, y_pred))

In [ ]:
#Muestra de variables mas importantes para el modelo
feature_importances_df = pd.DataFrame(
    {"feature": list(X.columns), "importance": classifier.feature_importances_}
).sort_values("importance", ascending=False)

# Mostrar
feature_importances_df

In [ ]:
#Grafico de variables mas importantes para el modelo

sns.barplot(x=feature_importances_df.feature, y=feature_importances_df.importance)
# agregar estiquestas

plt.xlabel("Feature Importance Score")
plt.ylabel("Features")
plt.title("Visualizing Important Features")
plt.xticks(
    rotation=45, horizontalalignment="right", fontweight="light", fontsize="x-large"
)
plt.show()